In [116]:
"""
@author: Matías Ramírez, Matías González
"""

'\n@author: Matías Ramírez, Matías González\n'

In [117]:
!pip install deap

In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
import pandas as pd
import numpy as np
from io import StringIO
from deap import base, creator, tools, algorithms

file_path = '/content/drive/MyDrive/vrptw_c101.txt'

with open(file_path, 'r') as file:
    data = file.read()


data_relevant = data.split('SERVICE TIME\n')[1]  # Extracting data after the header
data_stream = StringIO(data_relevant)  # Using StringIO to make it readable by pandas

df = pd.read_csv(data_stream, sep="\s+", header=None)
df.columns = ["CUST NO.", "XCOORD.", "YCOORD.", "DEMAND", "TIEMPO LISTO", "FECHA LÍMITE", "TIEMPO DE SERVICIO"]

df.head()

,CUST NO.,XCOORD.,YCOORD.,DEMAND,TIEMPO LISTO,FECHA LÍMITE,TIEMPO DE SERVICIO
0,0,40,50,0,0,1236,0
1,1,45,68,10,912,967,10
2,2,45,70,30,825,870,10
3,3,42,66,10,65,146,10
4,4,42,68,10,727,782,10


1.MOACS - ACO generico


In [122]:
import pandas as pd
import numpy as np

# Establecer semilla aleatoria para reproducibilidad
random_seed = np.random.randint(1, 1000)  # Genera un número aleatorio entre 1 y 1000
np.random.seed(random_seed)

# Calcular la distancia entre dos puntos
def calcular_distancia(punto1, punto2):
    return np.linalg.norm(np.array(punto1) - np.array(punto2))

# Calcular el costo de una ruta dadas las coordenadas de los clientes
def calcular_costo_ruta(ruta, coordenadas):
    distancias = np.linalg.norm(coordenadas[ruta[:-1]] - coordenadas[ruta[1:]], axis=1)
    return np.sum(distancias)

# Inicializar rutas para cada vehículo
def inicializar_rutas(num_clientes):
    return [[i] for i in range(1, num_clientes)]

# Seleccionar el cliente más cercano a uno dado
def seleccionar_cliente_mas_cercano(cliente_actual, clientes_restantes, coordenadas):
    if not clientes_restantes:
        return None
    ubicacion_actual = coordenadas[cliente_actual]
    distancias = np.linalg.norm(coordenadas[clientes_restantes] - ubicacion_actual, axis=1)
    cliente_mas_cercano = clientes_restantes[np.argmin(distancias)]
    return cliente_mas_cercano

# Verificar si se viola la ventana de tiempo
def se_viola_ventana_tiempo(tiempo_actual, distancia, tiempo_listo, fecha_limite, tiempo_servicio):
    return tiempo_actual + distancia + tiempo_servicio > fecha_limite

# Agregar un cliente a una ruta y verificar si se viola la ventana de tiempo
def agregar_cliente_a_ruta(ruta_actual, cliente, tiempo_actual, coordenadas, df, tolerancia_ventana_tiempo):
    if cliente is not None:
        distancia = calcular_distancia(coordenadas[ruta_actual[-1]], coordenadas[cliente])
        tiempo_servicio = df.at[cliente, 'TIEMPO DE SERVICIO']
        tiempo_listo = df.at[cliente, 'TIEMPO LISTO']
        fecha_limite = df.at[cliente, 'FECHA LÍMITE']

        if not se_viola_ventana_tiempo(tiempo_actual, distancia, tiempo_listo, fecha_limite, tiempo_servicio):
            ruta_actual.append(cliente)
            tiempo_actual = max(tiempo_actual + distancia + tiempo_servicio, tiempo_listo)
            return tiempo_actual, True
        else:
            return tiempo_actual, False
    else:
        return tiempo_actual, False

# Inicializar feromonas
def inicializar_feromonas(num_clientes):
    return np.ones((num_clientes, num_clientes)) / num_clientes

# Actualizar feromonas
def actualizar_feromonas(feromonas, rutas, costo_rutas, evaporation_rate=0.5):
    feromonas *= (1.0 - evaporation_rate)  # Evaporación
    for ruta, costo in zip(rutas, costo_rutas):
        for i in range(len(ruta) - 1):
            # Actualización de feromonas utilizando la ecuación clásica
            feromonas[ruta[i], ruta[i + 1]] += 1.0 / costo
            feromonas[ruta[i + 1], ruta[i]] += 1.0 / costo

# Construir solución de la hormiga
def construir_solucion_hormiga(feromonas, visibilidad, tolerancia_ventana_tiempo=0):
    num_clientes = feromonas.shape[0]
    coordenadas = df[['XCOORD.', 'YCOORD.']].values
    ruta_hormiga = [0]  # Comienza en el depósito
    tiempo_actual = 0

    while len(ruta_hormiga) < num_clientes:
        clientes_restantes = [c for c in range(num_clientes) if c not in ruta_hormiga]
        probabilidad = np.zeros(num_clientes)

        for cliente in clientes_restantes:
            probabilidad[cliente] = (feromonas[ruta_hormiga[-1], cliente] ** 1.0) * (visibilidad[ruta_hormiga[-1], cliente] ** 2.0)

        if np.random.rand() < 0.9:
            # Elegir la mejor (explotación)
            siguiente_cliente = np.argmax(probabilidad)
        else:
            # Elegir según la probabilidad proporcional (exploración)
            probabilidad /= np.sum(probabilidad)
            probabilidad[clientes_restantes] /= np.sum(probabilidad[clientes_restantes])
            siguiente_cliente = np.random.choice(clientes_restantes, p=probabilidad[clientes_restantes], replace=False)

        tiempo_actual, agregado = agregar_cliente_a_ruta(ruta_hormiga, siguiente_cliente, tiempo_actual, coordenadas, df, tolerancia_ventana_tiempo)

        if not agregado:
            # Si no se pudo agregar el cliente, regresar al depósito y continuar
            ruta_hormiga.append(0)
            tiempo_actual = 0

    return ruta_hormiga

# Calcular la visibilidad entre las ciudades
def calcular_visibilidad(coordenadas):
    num_clientes = coordenadas.shape[0]
    visibilidad = np.zeros((num_clientes, num_clientes))

    for i in range(num_clientes):
        for j in range(i + 1, num_clientes):
            dist = calcular_distancia(coordenadas[i], coordenadas[j])
            visibilidad[i, j] = 1.0 / dist
            visibilidad[j, i] = 1.0 / dist

    return visibilidad

def moaco(df, num_hormigas, num_iteraciones, tolerancia_ventana_tiempo=0,
          ALPHA=1.0, BETA=2.0, RHO=0.5, Q0=0.9):
    """
    Realiza la optimización de rutas utilizando el Algoritmo de Colonias de Hormigas (MOACO).

    Parámetros:
    - df: DataFrame que contiene la información de los clientes.
    - num_hormigas: Número de hormigas (soluciones) a construir en cada iteración.
    - num_iteraciones: Número de iteraciones del algoritmo.
    - tolerancia_ventana_tiempo: Tolerancia para la ventana de tiempo de los clientes.
    - ALPHA: Peso de la feromona en la elección de la próxima ciudad.
    - BETA: Peso de la visibilidad en la elección de la próxima ciudad.
    - RHO: Tasa de evaporación de la feromona.
    - Q0: Probabilidad de elegir la mejor ciudad (en lugar de basarse en probabilidades).

    Retorna:
    La mejor solución encontrada.
    """
    num_clientes = len(df)
    coordenadas = df[['XCOORD.', 'YCOORD.']].values

    feromonas = inicializar_feromonas(num_clientes)
    visibilidad = calcular_visibilidad(coordenadas)

    mejor_solucion_global = None
    mejor_costo_global = float('inf')

    for iteracion in range(num_iteraciones):
        soluciones_hormigas = []

        for _ in range(num_hormigas):
            solucion_hormiga = construir_solucion_hormiga(feromonas, visibilidad, tolerancia_ventana_tiempo)
            soluciones_hormigas.append(solucion_hormiga)

        costo_rutas_hormigas = [calcular_costo_ruta(solucion, coordenadas) for solucion in soluciones_hormigas]

        # Actualizar feromonas
        actualizar_feromonas(feromonas, soluciones_hormigas, costo_rutas_hormigas, evaporation_rate=RHO)


        # Actualizar mejor solución global
        mejor_indice = np.argmin(costo_rutas_hormigas)
        if costo_rutas_hormigas[mejor_indice] < mejor_costo_global:
            mejor_costo_global = costo_rutas_hormigas[mejor_indice]
            mejor_solucion_global = soluciones_hormigas[mejor_indice]


        promedio_feromonas = np.mean(feromonas)

        # Imprimir información
        print(f"Iteración {iteracion + 1}/{num_iteraciones} - Mejor Costo Global: {mejor_costo_global} - Promedio de Feromonas: {promedio_feromonas}")

    return mejor_solucion_global


# Imprimir la solución (rutas y costos)
# Agregar esto a la función imprimir_solucion
def imprimir_solucion(rutas, df):
    for i, ruta in enumerate(rutas):
        print(f"Ruta {i + 1}: Depósito -> ", end="")
        for cliente in ruta:
            print(f"{cliente} Tiempo Servicio: {df.at[cliente, 'TIEMPO DE SERVICIO']}, Ventana Tiempo: ({df.at[cliente, 'TIEMPO LISTO']}, {df.at[cliente, 'FECHA LÍMITE']})) -> ", end="")
        print("Depósito")
        print(f"Costo Ruta {i + 1}: {calcular_costo_ruta(ruta, df[['XCOORD.', 'YCOORD.']].values)}")
        print()


if __name__ == '__main__':
    print("Dataset")
    print(df)
    print()

    num_hormigas = 10
    num_iteraciones = 50
    tolerancia_ventana_tiempo = 30

    mejor_solucion = moaco(df, num_hormigas, num_iteraciones, tolerancia_ventana_tiempo)

    # Imprimir la mejor solución encontrada
    imprimir_solucion([mejor_solucion], df)

Dataset
     CUST NO.  XCOORD.  YCOORD.  DEMAND  TIEMPO LISTO  FECHA LÍMITE  \
0           0       40       50       0             0          1236   
1           1       45       68      10           912           967   
2           2       45       70      30           825           870   
3           3       42       66      10            65           146   
4           4       42       68      10           727           782   
..        ...      ...      ...     ...           ...           ...   
96         96       60       80      10            95           156   
97         97       60       85      30           561           622   
98         98       58       75      20            30            84   
99         99       55       80      10           743           820   
100       100       55       85      20           647           726   

     TIEMPO DE SERVICIO  
0                     0  
1                    10  
2                    10  
3                    10  
4        

2.MOACS se hace el intento de usar frente pareto para usar como objetivo tiempo y costo

In [127]:
import pandas as pd
import numpy as np

# Establecer semilla aleatoria para reproducibilidad
random_seed = np.random.randint(1, 1000)  # Genera un número aleatorio entre 1 y 1000
np.random.seed(random_seed)

# Calcular la distancia entre dos puntos
def calcular_distancia(punto1, punto2):
    return np.linalg.norm(np.array(punto1) - np.array(punto2))

# Calcular el costo de una ruta dadas las coordenadas de los clientes
def calcular_costo_ruta(ruta, coordenadas):
    distancias = np.linalg.norm(coordenadas[ruta[:-1]] - coordenadas[ruta[1:]], axis=1)
    return np.sum(distancias)

# Inicializar rutas para cada vehículo
def inicializar_rutas(num_clientes):
    return [[i] for i in range(1, num_clientes)]

# Seleccionar el cliente más cercano a uno dado
def seleccionar_cliente_mas_cercano(cliente_actual, clientes_restantes, coordenadas):
    if not clientes_restantes:
        return None
    ubicacion_actual = coordenadas[cliente_actual]
    distancias = np.linalg.norm(coordenadas[clientes_restantes] - ubicacion_actual, axis=1)
    cliente_mas_cercano = clientes_restantes[np.argmin(distancias)]
    return cliente_mas_cercano

# Verificar si se viola la ventana de tiempo
def se_viola_ventana_tiempo(tiempo_actual, distancia, tiempo_listo, fecha_limite, tiempo_servicio):
    return tiempo_actual + distancia + tiempo_servicio > fecha_limite

# Agregar un cliente a una ruta y verificar si se viola la ventana de tiempo
def agregar_cliente_a_ruta(ruta_actual, cliente, tiempo_actual, coordenadas, df, tolerancia_ventana_tiempo):
    if cliente is not None:
        distancia = calcular_distancia(coordenadas[ruta_actual[-1]], coordenadas[cliente])
        tiempo_servicio = df.at[cliente, 'TIEMPO DE SERVICIO']
        tiempo_listo = df.at[cliente, 'TIEMPO LISTO']
        fecha_limite = df.at[cliente, 'FECHA LÍMITE']

        if not se_viola_ventana_tiempo(tiempo_actual, distancia, tiempo_listo, fecha_limite, tiempo_servicio):
            ruta_actual.append(cliente)
            tiempo_actual = max(tiempo_actual + distancia + tiempo_servicio, tiempo_listo)
            return tiempo_actual, True
        else:
            return tiempo_actual, False
    else:
        return tiempo_actual, False

# Inicializar feromonas
def inicializar_feromonas(num_clientes):
    return np.ones((num_clientes, num_clientes)) / num_clientes

# Actualizar feromonas
def actualizar_feromonas(feromonas, rutas, costo_rutas, evaporation_rate=0.5):
    feromonas *= (1.0 - evaporation_rate)  # Evaporación
    for ruta, costo in zip(rutas, costo_rutas):
        for i in range(len(ruta) - 1):
            # Actualización de feromonas utilizando la ecuación clásica
            feromonas[ruta[i], ruta[i + 1]] += 1.0 / costo
            feromonas[ruta[i + 1], ruta[i]] += 1.0 / costo

# Construir solución de la hormiga
def construir_solucion_hormiga(feromonas, visibilidad, tolerancia_ventana_tiempo=0):
    num_clientes = feromonas.shape[0]
    coordenadas = df[['XCOORD.', 'YCOORD.']].values
    ruta_hormiga = [0]  # Comienza en el depósito
    tiempo_actual = 0

    while len(ruta_hormiga) < num_clientes:
        clientes_restantes = [c for c in range(num_clientes) if c not in ruta_hormiga]
        probabilidad = np.zeros(num_clientes)

        for cliente in clientes_restantes:
            probabilidad[cliente] = (feromonas[ruta_hormiga[-1], cliente] ** 1.0) * (visibilidad[ruta_hormiga[-1], cliente] ** 2.0)

        if np.random.rand() < 0.9:
            # Elegir la mejor (explotación)
            siguiente_cliente = np.argmax(probabilidad)
        else:
            # Elegir según la probabilidad proporcional (exploración)
            probabilidad /= np.sum(probabilidad)
            probabilidad[clientes_restantes] /= np.sum(probabilidad[clientes_restantes])
            siguiente_cliente = np.random.choice(clientes_restantes, p=probabilidad[clientes_restantes], replace=False)

        tiempo_actual, agregado = agregar_cliente_a_ruta(ruta_hormiga, siguiente_cliente, tiempo_actual, coordenadas, df, tolerancia_ventana_tiempo)

        if not agregado:
            # Si no se pudo agregar el cliente, regresar al depósito y continuar
            ruta_hormiga.append(0)
            tiempo_actual = 0

    return ruta_hormiga

# Calcular la visibilidad entre las ciudades
def calcular_visibilidad(coordenadas):
    num_clientes = coordenadas.shape[0]
    visibilidad = np.zeros((num_clientes, num_clientes))

    for i in range(num_clientes):
        for j in range(i + 1, num_clientes):
            dist = calcular_distancia(coordenadas[i], coordenadas[j])
            visibilidad[i, j] = 1.0 / dist
            visibilidad[j, i] = 1.0 / dist

    return visibilidad

# Modified function to calculate the time for a route
def calcular_tiempo_ruta(ruta, coordenadas, df):
    # Assuming time is the sum of service times for each customer in the route
    tiempo_servicio = sum(df.at[cliente, 'TIEMPO DE SERVICIO'] for cliente in ruta[1:-1])
    return tiempo_servicio

def calcular_pareto_front(costos, tiempos):
    combined_objectives = np.column_stack((costos, tiempos))

    sorted_indices = np.lexsort(combined_objectives.T)

    # Keep only the non-dominated solutions
    pareto_front = [sorted_indices[0]]
    for i in range(1, len(sorted_indices)):
        if combined_objectives[sorted_indices[i], 1] < combined_objectives[pareto_front[-1], 1]:
            pareto_front.append(sorted_indices[i])

    return pareto_front

# Modified moaco function to return Pareto front solutions
def moaco(df, num_hormigas, num_iteraciones, tolerancia_ventana_tiempo=0,
          ALPHA=1.0, BETA=2.0, RHO=0.5, Q0=0.9):
    num_clientes = len(df)
    coordenadas = df[['XCOORD.', 'YCOORD.']].values

    feromonas = inicializar_feromonas(num_clientes)
    visibilidad = calcular_visibilidad(coordenadas)

    pareto_front_solutions = []

    mejor_solucion_global = None
    mejor_costo_global = float('inf')

    for iteracion in range(num_iteraciones):
        soluciones_hormigas = []

        for _ in range(num_hormigas):
            solucion_hormiga = construir_solucion_hormiga(feromonas, visibilidad, tolerancia_ventana_tiempo)
            soluciones_hormigas.append(solucion_hormiga)

        costo_rutas_hormigas = [calcular_costo_ruta(solucion, coordenadas) for solucion in soluciones_hormigas]
        tiempo_rutas_hormigas = [calcular_tiempo_ruta(solucion, coordenadas, df) for solucion in soluciones_hormigas]

        pareto_front = calcular_pareto_front(costo_rutas_hormigas, tiempo_rutas_hormigas)
        pareto_front_solutions.extend([soluciones_hormigas[idx] for idx in pareto_front])

        # Update pheromones
        actualizar_feromonas(feromonas, soluciones_hormigas, costo_rutas_hormigas, evaporation_rate=RHO)

        for idx in pareto_front:
            if costo_rutas_hormigas[idx] < mejor_costo_global:
                mejor_costo_global = costo_rutas_hormigas[idx]
                mejor_solucion_global = soluciones_hormigas[idx]

        promedio_feromonas = np.mean(feromonas)

        # Print information
        print(f"Iteración {iteracion + 1}/{num_iteraciones} - Mejor Costo Global: {mejor_costo_global} - Promedio de Feromonas: {promedio_feromonas}")

    return pareto_front_solutions


# Imprimir la solución (rutas y costos)
# Agregar esto a la función imprimir_solucion
def imprimir_solucion(rutas, df):
    for i, ruta in enumerate(rutas):
        print(f"Ruta {i + 1}: Depósito -> ", end="")
        for cliente in ruta:
            print(f"{cliente} Tiempo Servicio: {df.at[cliente, 'TIEMPO DE SERVICIO']}, Ventana Tiempo: ({df.at[cliente, 'TIEMPO LISTO']}, {df.at[cliente, 'FECHA LÍMITE']})) -> ", end="")
        print("Depósito")
        print(f"Costo Ruta {i + 1}: {calcular_costo_ruta(ruta, df[['XCOORD.', 'YCOORD.']].values)}")
        print()


if __name__ == '__main__':
    print("Dataset")
    print(df)
    print()

    num_hormigas = 10
    num_iteraciones = 20
    tolerancia_ventana_tiempo = 30

    pareto_front_solutions = moaco(df, num_hormigas, num_iteraciones, tolerancia_ventana_tiempo)

    # Print Pareto front solutions
    for i, solution in enumerate(pareto_front_solutions):
        print(f"Pareto Front Solution {i + 1}")
        imprimir_solucion([solution], df)

Dataset
     CUST NO.  XCOORD.  YCOORD.  DEMAND  TIEMPO LISTO  FECHA LÍMITE  \
0           0       40       50       0             0          1236   
1           1       45       68      10           912           967   
2           2       45       70      30           825           870   
3           3       42       66      10            65           146   
4           4       42       68      10           727           782   
..        ...      ...      ...     ...           ...           ...   
96         96       60       80      10            95           156   
97         97       60       85      30           561           622   
98         98       58       75      20            30            84   
99         99       55       80      10           743           820   
100       100       55       85      20           647           726   

     TIEMPO DE SERVICIO  
0                     0  
1                    10  
2                    10  
3                    10  
4        